In [ ]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn import preprocessing
import numpy as log1p
from sklearn.preprocessing import PolynomialFeatures


In [ ]:
df=pd.read_csv("../input/30days-folds/train_folds.csv")
df_test= pd.read_csv("../input/30-days-of-ml/test.csv")
sample_submission=pd.read_csv("../input/30-days-of-ml/sample_submission.csv")

useful_features=[c for c in df.columns if c not in("id","target","kfold")]
object_cols= [col for col in useful_features if "cat"in col]

df_test= df_test[useful_features]

for col in object_cols:
    temp_df=[]
    temp_test_feat=None
    for fold in range(5):
        xtrain= df[df.kfold != fold].reset_index(drop=True)
        xvalid= df[df.kfold== fold].reset_index(drop= True)
        


        
        feat= xtrain.groupby(col)["target"].agg('count')
        feat= feat.to_dict()
        
        xvalid.loc[:, f"tar_enc_{col}"]= xvalid[col].map(feat)
        temp_df.append(xvalid)
        if temp_test_feat is None:
            temp_test_feat=df_test[col].map(feat)
        else:
             temp_test_feat=df_test[col].map(feat)
    temp_test_feat/=5
    df_test.loc[:, f"tar_enc_{col}"]= temp_test_feat
    df=pd.concat(temp_df)
      

In [ ]:
useful_features=[c for c in df.columns if c not in("id","target","kfold")]
object_cols= [col for col in useful_features if col.startswith("cat") ]

df_test= df_test[useful_features]

In [ ]:
final_predictions=[]
scores=[]
for fold in range(5):
    xtrain= df[df.kfold != fold].reset_index(drop=True)
    xvalid= df[df.kfold== fold].reset_index(drop= True)
    xtest= df_test.copy()
    ytrain= xtrain.target
    yvalid= xvalid.target
    
    xtrain= xtrain[useful_features]
    xvalid= xvalid[useful_features]
    
    
    ordinal_encoder= preprocessing.OrdinalEncoder()
    xtrain[object_cols]= ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols]= ordinal_encoder.transform(xvalid[object_cols])
    xtest[object_cols]= ordinal_encoder.transform(xtest[object_cols])
    
   
    
    
    model= XGBRegressor(random_state=fold,tree_method='gpu_hist',gpu_id=0, predictor="gpu_predictor")
    model.fit(xtrain, ytrain)
    preds_valid= model.predict(xvalid)
    test_preds=model.predict(xtest)
    final_predictions.append(test_preds)
    rmse= mean_squared_error(yvalid, preds_valid)
    print(fold,rmse)
    scores.append(rmse)
print(np.mean(scores), np.std(scores))

In [ ]:
df.shape

In [ ]:
sample_submission.shape

In [ ]:
final_predictions[0]

In [ ]:
sample_submission["target"] = final_predictions[0]

In [ ]:
sample_submission.to_csv('submission_kaggle.csv', index=False)